<a href="https://colab.research.google.com/github/ArmFriiz/Dicoding-Submission-FDL/blob/main/Analisis%20Sentimen/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Muhammad Faris Akbar**
<br></br>
**Fundamental Deep Learning - Sentimen Analisis Shopee**

In [25]:
!pip install google-play-scraper transformers

In [26]:
import sys
import os
import pandas as pd
import re

In [27]:
# try:
#     # Jika dijalankan sebagai file .py
#     base_path = os.path.dirname(os.path.abspath(__file__))
# except NameError:
#     # Jika dijalankan di Jupyter/Interactive
#     base_path = os.getcwd()

# parent_dir = os.path.abspath(os.path.join(base_path, '..'))
# sys.path.append(parent_dir)

# print("Base Path:", base_path)
# print("Parent Directory:", parent_dir)

# from Helper import *

In [28]:
from google_play_scraper import reviews, Sort
from transformers import pipeline
from tqdm import tqdm

**Helper Function**

In [29]:
def cek_nan(data):
  """
  Function yang digunakan untuk memeriksa nilai nan dari data
  input   : data
  output  : nan_info (DataFrame)
  return  : nan_info (DataFrame)
  """
  nan_info = pd.DataFrame(data.isna().sum().sort_values(ascending=False), columns=['Jumlah Nilai Missing'])

  if nan_info['Jumlah Nilai Missing'].sum() == 0:
    print("Tidak ada nilai missing")
    return None
  else:
    nan_info['Persentase Nilai Missing (%)'] = ((nan_info['Jumlah Nilai Missing'] / len(data)) * 100).round(3)

    # return data dengan nilai missing, apabila ingin mereturn keseluruhan data maka ubah menjadi return nan_info
    return nan_info[nan_info['Jumlah Nilai Missing'] > 0]

In [30]:
def visualize_row_with_nan(data, method='all', columns=None):
  """
  Procedure yang digunakan untuk menvisualisasikan baris yang mengandung NaN berdasarkan metode yang dipilih
  input   : data, method, columns
  output  : baris_nan (DataFrame)
  """
  try:
    if method not in ['all', 'column', 'columns']:
      raise ValueError("Metode tidak valid. Pilih salah satu dari 'all', 'column', 'columns'.")

    if method == 'all':
      display(data[data.isna().any(axis=1)])

    elif method == 'column':
      if not isinstance(columns, str):
          raise TypeError("Untuk metode 'column', argumen 'columns' harus berupa String.")
      if columns not in data.columns:
          raise KeyError(f"Kolom '{columns}' tidak ditemukan dalam DataFrame.")
      display(data[data[columns].isna()])

    elif method == 'columns':
      if not isinstance(columns, list):
          raise TypeError("Untuk metode 'columns', argumen 'columns' harus berupa list.")
      for column in columns:
          if column not in data.columns:
              raise KeyError(f"Kolom '{column}' tidak ditemukan dalam DataFrame.")
      display(data[data[columns].isna().all(axis=1)])

  except (TypeError, KeyError) as e:
    print(f"Kesalahan dalam memproses data: {e}")

In [31]:
def visualize_row_with_duplicated(data):
  """
  Procedure yang digunakan untuk menvisualisasikan baris yang mengandung data duplikat
  input   : data
  output  : baris_duplikat (DataFrame)
  """
  duplicated = data.duplicated().sum()

  if duplicated > 0:
    print("Jumlah Data Duplikat :", duplicated)
    all_duplicates = data[data.duplicated(keep=False)]

    duplicate_indices = all_duplicates.groupby(list(all_duplicates.columns)).groups
    print("Pasangan Data Duplikat :")
    for group_indices in duplicate_indices.values():
      if len(group_indices) > 1:
        display(data.iloc[list(group_indices)])
        print("\n")
  else:
    print("Tidak ada data duplikat")

**Main Section**

In [32]:
def scrape_google_play(app_id, total_count, country='id', lang='id'):
    """
    Fungsi untuk melakukan scraping ulasan dari Google Play Store.

    app_id (str): ID aplikasi di Play Store (contoh: 'com.shopee.id')
    total_count (int): Target jumlah data yang ingin diambil
    country (str): Kode negara (default Indonesia 'id')
    lang (str): Bahasa ulasan (default Indonesia 'id')
    """
    print(f"Proses scraping untuk aplikasi: {app_id}...")

    result, continuation_token = reviews(
        app_id,
        lang=lang,
        country=country,
        sort=Sort.NEWEST, # Sort.NEWEST digunakan untuk mendapat data terbaru
        count=total_count,
        filter_score_with=None # Mengambil semua rating (1-5)
    )

    print(f"Berhasil mengambil {len(result)} data mentah.")

    # Konversi hasil scraping ke dalam DataFrame (Tabel)
    df = pd.DataFrame(result)

    return df

In [33]:
TARGET_APP = 'com.shopee.id'
JUMLAH_DATA = 5000

df_ulasan = scrape_google_play(TARGET_APP, JUMLAH_DATA)

Proses scraping untuk aplikasi: com.shopee.id...
Berhasil mengambil 5000 data mentah.


In [34]:
# df_ulasan = pd.read_csv('https://raw.githubusercontent.com/ArmFriiz/Dicoding-Submission-FDL/refs/heads/main/Analisis%20Sentimen/dataset_ulasan_playstore.csv')

In [35]:
df_ulasan.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ceda1c3b-35aa-465d-a7db-2a5b281d8fcb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Selama ini si saya suka berbelanja dg shopie d...,5,0,2.11.13,2026-01-28 07:40:49,"Hai kak Aprilia Lesmana , mohon maaf atas keti...",2026-01-28 08:11:35,2.11.13
1,11a01d47-6aa3-4cdf-820b-f1b8ce01f139,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,nice,5,0,3.67.25,2026-01-28 07:38:00,"Hai kak Sirod Judi , makasih ya bintang dan re...",2026-01-28 08:08:42,3.67.25
2,6f800dc1-1214-4bc1-a2b3-4fff93a6880c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"pokonamah mantap lah sesuai pesanan,👍",5,0,2.99.07,2026-01-28 07:37:52,"Hai Kak Duchin Duchin, makasih ya buat review ...",2026-01-28 08:07:54,2.99.07
3,7ce7926e-a792-4bdd-9fce-fe7fafd4b32b,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,puas sekali memudahkan untuk belanja tanpa bua...,5,0,3.66.27,2026-01-28 07:37:46,Hi Kak Muhammad Fatkhurrizky makasih bgt ya re...,2026-01-28 08:08:25,3.66.27
4,dbb67844-a290-4bca-95ed-45589ac18be8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,barang saya hilang pengembalian dana ribet buk...,1,0,3.67.25,2026-01-28 07:36:24,"Haii kak Yourdhan Yoga, maaf ya udh bikin km k...",2026-01-28 08:37:11,3.67.25


In [36]:
df_ulasan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              5000 non-null   object        
 1   userName              5000 non-null   object        
 2   userImage             5000 non-null   object        
 3   content               5000 non-null   object        
 4   score                 5000 non-null   int64         
 5   thumbsUpCount         5000 non-null   int64         
 6   reviewCreatedVersion  3870 non-null   object        
 7   at                    5000 non-null   datetime64[ns]
 8   replyContent          4832 non-null   object        
 9   repliedAt             4832 non-null   datetime64[ns]
 10  appVersion            3870 non-null   object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 429.8+ KB


In [37]:
df_ulasan.describe(include='all')

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
count,5000,5000,5000,5000,5000.000000,5000.000000,3870,5000,4832,4832,3870
unique,5000,496,498,4034,NaN,NaN,139,NaN,4706,NaN,139
top,8559618d-9db7-4212-8fa9-55916f209439,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,NaN,NaN,3.66.27,NaN,"Hai Kak , makasih ya buat penilaiannya, semoga...",NaN,3.66.27
freq,1,4504,4503,164,NaN,NaN,2272,NaN,72,NaN,2272
mean,NaN,NaN,NaN,NaN,4.114400,2.559000,NaN,2026-01-25 07:27:17.913600,NaN,2025-12-21 19:58:12.074503424,NaN
min,NaN,NaN,NaN,NaN,1.000000,0.000000,NaN,2026-01-22 10:56:00,NaN,2020-11-18 09:06:01,NaN
25%,NaN,NaN,NaN,NaN,4.000000,0.000000,NaN,2026-01-23 22:38:46.750000128,NaN,2026-01-23 14:07:54.249999872,NaN
50%,NaN,NaN,NaN,NaN,5.000000,0.000000,NaN,2026-01-25 06:45:19.500000,NaN,2026-01-25 03:22:02.500000,NaN
75%,NaN,NaN,NaN,NaN,5.000000,0.000000,NaN,2026-01-26 14:50:34.750000128,NaN,2026-01-26 13:28:38.500000,NaN
max,NaN,NaN,NaN,NaN,5.000000,8284.000000,NaN,2026-01-28 07:40:49,NaN,2026-01-28 12:57:14,NaN


**Cek Validitas dan Kebersihan Data**

In [38]:
cek_nan(df_ulasan)

,Jumlah Nilai Missing,Persentase Nilai Missing (%)
reviewCreatedVersion,1130,22.60
appVersion,1130,22.60
replyContent,168,3.36
repliedAt,168,3.36


In [39]:
visualize_row_with_nan(df_ulasan)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
8,45b1a79b-aa5f-4e1d-bcd1-8b357257003f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,uang jualan selalu di tilep shopee,1,0,None,2026-01-28 07:35:46,"Hi kak Nugroho Nusantoro, maaf banget ya udah ...",2026-01-28 08:10:20,None
9,cb767cb6-76ef-41ed-aae9-0e6ac5a2443a,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,barang tiba tepat waktu dan ordersnnya sesuai,5,0,None,2026-01-28 07:35:33,Hallo kak Suhatim Suhatim. Makasih yaa buat re...,2026-01-28 08:07:43,None
12,c8dc9394-0a8b-4006-af40-08020600cc07,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,sangat mantap.mempermudah urusan financial saya,5,0,None,2026-01-28 07:33:03,"Hi kak Syamsoel Boy, makasih ya buat bintang 5...",2026-01-28 08:08:41,None
16,27454294-a5f9-437c-82e8-fedd143cd01d,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,suda lama pakai Shopee tapi sering error mau b...,1,0,None,2026-01-28 07:32:09,"Hi kak Rema wanti, maaf ya udah bikin km ga ny...",2026-01-28 08:09:38,None
19,070fb11e-3c48-4a96-8c8e-cdb3ec0c562c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus,5,0,None,2026-01-28 07:27:32,Hi Kak Aep saepudin Saepudin makasih ya untuk ...,2026-01-28 08:09:26,None
...,...,...,...,...,...,...,...,...,...,...,...
4974,ce0fb20c-e105-4d17-b0c1-49dd8e97784a,Indri Jon istovel,https://play-lh.googleusercontent.com/a-/ALV-U...,Shopee mantab,5,0,None,2026-01-22 11:17:35,"Hai Kak , makasih ya buat penilaiannya, semoga...",2026-01-22 13:35:58,None
4977,d28174e4-258c-49a8-91c0-bc24d54aa64d,Helmi Alfariz,https://play-lh.googleusercontent.com/a/ACg8oc...,sangat memuaskan dalam berbelanja segala kebut...,5,0,None,2026-01-22 11:15:47,Hallo kak Helmi Alfariz. Makasih yaa buat revi...,2026-01-22 13:40:40,None
4979,68ab749f-7f5d-40fc-ba36-cd61c572bbf3,Indriyani 17,https://play-lh.googleusercontent.com/a/ACg8oc...,iklanny ada di mna' gedeg bngt tiba' masuk k a...,3,0,None,2026-01-22 11:15:29,"Hi kak Indriyani 17, mohon maaf ya kak atas ke...",2026-01-22 13:36:48,None
4985,4ca309b3-da0b-4ae0-82d9-fc0b9639d49f,Wisonggeni,https://play-lh.googleusercontent.com/a-/ALV-U...,👍,5,0,None,2026-01-22 11:09:29,"Hai Kak , makasih ya buat penilaiannya, semoga...",2026-01-22 13:38:56,None


In [40]:
visualize_row_with_duplicated(df_ulasan)

Tidak ada data duplikat


**Filter kolom content dan score untuk konten data**

In [41]:
df = df_ulasan[['content', 'score']]

In [42]:
visualize_row_with_duplicated(df)

Jumlah Data Duplikat : 919
Pasangan Data Duplikat :


,content,score
418,Alhamdulillah,5
820,Alhamdulillah,5
1823,Alhamdulillah,5
3003,Alhamdulillah,5


,content,score
226,Bagus,5
657,Bagus,5
713,Bagus,5
1436,Bagus,5
1733,Bagus,5
1802,Bagus,5
1919,Bagus,5
3197,Bagus,5
3688,Bagus,5
3755,Bagus,5


,content,score
348,Baik,5
1557,Baik,5
3496,Baik,5
4312,Baik,5


,content,score
1513,Good,5
1677,Good,5
3192,Good,5
3630,Good,5
4641,Good,5


...

,content,score
3066,top markotop,5
3697,top markotop,5
4789,top markotop,5


,content,score
1829,very good,5
2246,very good,5


,content,score
1154,👍,5
1366,👍,5
2375,👍,5
3522,👍,5
3770,👍,5
4148,👍,5
4284,👍,5
4451,👍,5
4985,👍,5


,content,score
3099,👍🏻,5
3107,👍🏻,5


,content,score
1210,👍👍👍,5
1257,👍👍👍,5
2408,👍👍👍,5
3079,👍👍👍,5
3450,👍👍👍,5


In [43]:
df.drop_duplicates(inplace=True)

/tmp/ipython-input-3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [44]:
visualize_row_with_duplicated(df)

Tidak ada data duplikat


In [45]:
print(f"Ukuran Data Setelah Pembersihan NaN dan Duplicated: {df.shape}")

Ukuran Data Setelah Pembersihan NaN dan Duplicated: (4081, 2)


**Soft Cleaning Data**

In [46]:
def cleaning_untuk_labeling(text):
    text = str(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # Hapus URL
    text = re.sub(r'<.*?>', '', text) # Hapus HTML tags
    text = re.sub(r'@[A-Za-z0-9_]+', '', text) # Hapus Mentions (@user)
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # Hapus hashtag
    text = re.sub(r'(.)\1{2,}', r'\1\1', text) # Hapus kata berulang yang muncul lebih dari 2x
    text = re.sub(r'[a-zA-Z]+\d+\w*|\w*\d+[a-zA-Z]+', '', text) # Hapus kombinasi angka dan huruf seperti m4ndi, 4yam, dll
    text = re.sub(r'\b\d{7,}\b', '', text) # Hapus angka yang panjangnya lebih dari 7
    text = ' '.join(text.split())

    return text

In [47]:
df['soft_clean_content'] = df['content'].apply(cleaning_untuk_labeling)

/tmp/ipython-input-3141896218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['soft_clean_content'] = df['content'].apply(cleaning_untuk_labeling)


In [48]:
df.head(5)

,content,score,soft_clean_content
0,Selama ini si saya suka berbelanja dg shopie d...,5,Selama ini si saya suka berbelanja dg shopie d...
1,nice,5,nice
2,"pokonamah mantap lah sesuai pesanan,👍",5,"pokonamah mantap lah sesuai pesanan,👍"
3,puas sekali memudahkan untuk belanja tanpa bua...,5,puas sekali memudahkan untuk belanja tanpa bua...
4,barang saya hilang pengembalian dana ribet buk...,1,barang saya hilang pengembalian dana ribet buk...


In [49]:
cek_nan(df)

Tidak ada nilai missing


In [50]:
visualize_row_with_nan(df)

,content,score,soft_clean_content


In [51]:
df.dropna(inplace=True)

/tmp/ipython-input-1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


**Labeling Data**

In [52]:
# def labeling_data(df):
#     """
#     Fungsi untuk memberikan label otomatis berdasarkan skor bintang.
#     Menggunakan logika:
#     1-2 Bintang = Negatif
#     3 Bintang   = Netral
#     4-5 Bintang = Positif
#     """
#     def get_sentiment(score):
#         if score <= 2:
#             return 'Negatif'
#         elif score == 3:
#             return 'Netral'
#         else:
#             return 'Positif'

#     # Terapkan fungsi get_sentiment ke kolom 'score'
#     df['label'] = df['score'].apply(get_sentiment)

#     return df

In [53]:
# print("Labeling data berdasarkan skor bintang")
# df_labeled = labeling_data(df)

In [54]:
def label_with_indobert(df):
  pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

  nlp = pipeline(
      "sentiment-analysis",
      model=pretrained_name,
      tokenizer=pretrained_name,
      truncation=True, # Potong teks jika terlalu panjang (>512 kata)
      max_length=512
    )

  labels = []
  scores = []

  for text in tqdm(df['soft_clean_content']):
    try:
      result = nlp(text)[0] # Prediksi sentimen

      label = result['label'] # output: 'positive', 'neutral', 'negative'

      # Mapping ulang ke format Bahasa Indonesia
      label_map = {
        'positive': 'Positif',
        'neutral': 'Netral',
        'negative': 'Negatif'
      }
      labels.append(label_map.get(label, label))
      scores.append(result['score'])

    except Exception as e:
      print(f"Error pada teks: {text}")
      labels.append("Netral")
      scores.append(0.0)

  df['sentiment_label'] = labels
  df['confidence_score'] = scores

  return df

In [ ]:
df_labeled = label_with_indobert(df)

In [ ]:
df_labeled.head(5)

**Pemeriksaan Distribusi Data, memastikan apakah terdapat imbalance atau tidak**

In [ ]:
print("Distribusi Data per Kelas:")
print(df_labeled['sentiment_label'].value_counts())

**Konversi ke csv untuk mempermudah dalam pembersihan data lebih lanjut**

In [ ]:
nama_file = 'dataset_ulasan_playstore.csv'
df_labeled.to_csv(nama_file, index=False)

print(f"Selesai! Data berhasil disimpan ke '{nama_file}'")
print(f"Total data: {len(df_labeled)}")